# 🎵 4-Stem Music Source Separation Training

Train U-Net model để tách nhạc thành 4 stems: **Vocals, Drums, Bass, Other**

---

## Bước 1: Kiểm tra GPU

In [ ]:
# Kiểm tra GPU có sẵn hay không
!nvidia-smi

## Bước 2: Mount Google Drive (để lưu checkpoint)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Bước 3: Clone code từ GitHub

**Lưu ý:** Thay `<your-username>` bằng username GitHub của bạn

In [ ]:
# Clone repository
!git clone https://github.com/pvhoaicntt2211/DoAn4MSS.git
%cd DoAn4MSS

## Bước 4: Cài thư viện

In [ ]:
!pip install -q torch numpy librosa tqdm soundfile requests

## Bước 5: Tự động tải MUSDB18-HQ dataset

⚠️ **Lưu ý:** Dataset MUSDB18-HQ khá lớn (~30GB). Quá trình tải có thể mất 30-60 phút tùy tốc độ mạng.

Nếu đã có data sẵn trên Drive, có thể skip bước này và copy data từ Drive thay vì download lại.

In [ ]:
# Tải và giải nén MUSDB18-HQ tự động
!python scripts/download_musdb18.py --output data/

### Phương án 2: Sử dụng data có sẵn trên Drive (nếu đã tải trước)

Nếu đã có MUSDB18-HQ trên Drive, uncomment và chạy cell dưới thay vì download:

In [ ]:
# Uncomment các dòng dưới nếu muốn dùng data từ Drive
# !cp -r /content/drive/MyDrive/MUSDB18_data/train data/train
# !cp -r /content/drive/MyDrive/MUSDB18_data/test data/test
# print("✅ Đã copy data từ Drive")

## Bước 6: Kiểm tra dữ liệu

Verify rằng data đã được tải đúng với cấu trúc 4-stem

In [ ]:
import os

# Kiểm tra số lượng bài hát
train_songs = [d for d in os.listdir('data/train') if os.path.isdir(os.path.join('data/train', d))]
test_songs = [d for d in os.listdir('data/test') if os.path.isdir(os.path.join('data/test', d))]

print(f"📊 Dataset Statistics:")
print(f"  Train songs: {len(train_songs)}")
print(f"  Test songs: {len(test_songs)}")
print()

# Kiểm tra 1 bài mẫu
if train_songs:
    sample_song = train_songs[0]
    song_path = os.path.join('data/train', sample_song)
    files = os.listdir(song_path)
    print(f"📁 Sample song: {sample_song}")
    print(f"  Files: {sorted(files)}")
    
    # Verify có đủ 4 stems + mixture
    required_files = ['vocals.wav', 'drums.wav', 'bass.wav', 'other.wav', 'mixture.wav']
    has_all = all(f in files for f in required_files)
    if has_all:
        print("  ✅ All 4 stems + mixture present")
    else:
        print("  ⚠️ Warning: Missing some stem files")
        missing = [f for f in required_files if f not in files]
        print(f"  Missing: {missing}")

## Bước 7: Train mô hình 4-stem

**Lưu ý:** 
- Training với 4 stems sẽ tốn nhiều memory hơn 2-stem
- Có thể giảm batch-size nếu gặp lỗi Out of Memory
- Checkpoint sẽ lưu vào `/content/DoAn4MSS/checkpoints/`
- Nhớ backup về Drive sau mỗi vài epoch để không mất data khi session timeout

In [ ]:
# Train 50 epoch với GPU
# Điều chỉnh batch-size nếu cần (giảm xuống 8 hoặc 4 nếu OOM)
!python train.py \
    --train-dir data/train \
    --valid-dir data/test \
    --epochs 50 \
    --batch-size 8 \
    --num-workers 2

## Bước 8: Backup checkpoint về Drive

**Quan trọng:** Luôn backup checkpoint về Drive sau khi train để tránh mất dữ liệu khi Colab session kết thúc.

In [ ]:
# Copy checkpoint về Drive để không mất khi session kết thúc
!mkdir -p /content/drive/MyDrive/MSS_4stem_checkpoints
!cp -r checkpoints/* /content/drive/MyDrive/MSS_4stem_checkpoints/
print("✅ Đã backup checkpoint vào Drive")

## Bước 9: Test inference (4 stems)

Test model với 1 bài để xem kết quả 4-stem separation

In [ ]:
import os

# Lấy 1 bài test làm mẫu
test_songs = [d for d in os.listdir('data/test') if os.path.isdir(os.path.join('data/test', d))]
if test_songs:
    test_song = test_songs[0]
    test_file = f"data/test/{test_song}/mixture.wav"
    
    print(f"Testing with: {test_song}")
    
    # Tách 4 stems
    !python inference.py "{test_file}" \
        --checkpoint checkpoints/best_model.pth \
        --outdir test_output
    
    print("\n✅ Separation complete! Output files:")
    !ls -lh test_output/
else:
    print("⚠️ No test songs found")

## Bước 10: Visualize và nghe kết quả

Nghe từng stem đã được tách

In [ ]:
from IPython.display import Audio, display
import glob

# Tìm tất cả output files
output_files = sorted(glob.glob('test_output/*.wav'))

if output_files:
    for audio_file in output_files:
        stem_name = audio_file.split('_')[-1].replace('.wav', '')
        print(f"\n🎵 {stem_name.upper()}:")
        display(Audio(audio_file, rate=44100))
else:
    print("⚠️ No output files found")

## Bước 11: Visualize spectrograms (Optional)

So sánh spectrograms của các stems

In [ ]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np

# Load và visualize spectrograms
stems = ['vocals', 'drums', 'bass', 'other']
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('4-Stem Spectrograms', fontsize=16)

for idx, stem in enumerate(stems):
    row = idx // 2
    col = idx % 2
    
    # Find the output file for this stem
    stem_files = glob.glob(f'test_output/*_{stem}.wav')
    if stem_files:
        audio, sr = librosa.load(stem_files[0], sr=44100, mono=True)
        
        # Compute spectrogram
        S = librosa.stft(audio)
        S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
        
        # Plot
        img = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz', 
                                       ax=axes[row, col], cmap='viridis')
        axes[row, col].set_title(f'{stem.capitalize()} Spectrogram')
        axes[row, col].set_ylabel('Frequency (Hz)')
        axes[row, col].set_xlabel('Time (s)')
        fig.colorbar(img, ax=axes[row, col], format='%+2.0f dB')

plt.tight_layout()
plt.show()

print("✅ Visualization complete!")

## 🎉 Hoàn tất!

Model 4-stem đã được train và test thành công!

### Các bước tiếp theo:

1. **Download checkpoint** từ Drive về máy local:
   - Vào `/content/drive/MyDrive/MSS_4stem_checkpoints/`
   - Download `best_model.pth`
   - Copy vào thư mục `checkpoints/` trong project

2. **Chạy inference local**:
   ```bash
   python inference.py input.wav --checkpoint checkpoints/best_model.pth
   ```

3. **Chạy web app**:
   ```bash
   python app.py
   ```
   Truy cập http://localhost:5000 để sử dụng giao diện web

### Tips:
- Có thể tiếp tục train thêm nếu loss chưa hội tụ
- Điều chỉnh `STEM_WEIGHTS` trong `config.py` để ưu tiên các stem khác nhau
- Test với nhiều loại nhạc khác nhau để đánh giá chất lượng model